Python Threat Hunting _ Multiple malicious IOCs list checker

In [ ]:
#!pip3 install selenium
#!pip3 install webdriver_manager
from selenium.webdriver.chrome.service import Service               # manage Chrome driver process
from selenium.webdriver.chrome.options import Options               # configure Chrome settings before launching
from selenium import webdriver                                      # main browser controller
from selenium.webdriver.common.by import By                         # define how to find elements on web page
from webdriver_manager.chrome import ChromeDriverManager
import validators
import os
import time

# load iocs.txt file, which is a list of IPs, domains, and hashes, separated by a new line.
def load_iocs(filename):
    iocs = []

    # open file in read mode
    with open(filename, "r") as file:
        for line in file.readlines():
            iocs.append(line.strip())
    
    # print output
    print(f"Loaded {len(iocs)} IOCs.")
    return iocs

# extract details from VirusTotal page
def extract_vt_details(driver, ioc):
    try:
        time.sleep(3)        # Wait for page to load        
        body_text = driver.find_element(By.TAG_NAME, "body").text        # Get all visible text from the page
        import re       # use regular expressions library for pattern-matching detection
        
        # Look for "X / Y" or "X/Y" (detection ratio)
        detection_match = re.search(r'(\d+)\s*/\s*(\d+)', body_text)
        if detection_match:
            malicious = detection_match.group(1)
            total = detection_match.group(2)
            return f"{ioc} - {malicious}/{total} detections!"
        
        return f"{ioc} - No detection info found"        # If no detection ratio found, just return basic info
    
    except Exception as e:
        return f"{ioc} - Error: {str(e)}"

# run IOC list through VT lookups
def run(iocs):
    # create Chrome web driver to run VT
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    count = 1
    results = []

    for ioc in iocs:
        print(f"Processing {count}/{len(iocs)}")

        driver.switch_to.new_window("tab")      # add a new tab to browser window, one window with multiple tags as the result.
        ioc_type = None
        
        # filter based on IOC type by using the validators package. IOC with 32 characters in length is MD5 hash, 40 is SHA1 hash and 64 is SHA256 hash.
        if validators.ip_address.ipv4(ioc):
            driver.get(f"https://www.virustotal.com/gui/ip-address/{ioc}")
        elif validators.domain(ioc):
            driver.get(f"https://www.virustotal.com/gui/domain/{ioc}")
        elif len(ioc) == 32:    # MD5 hash
            driver.get(f"https://www.virustotal.com/gui/file/{ioc}")
        elif len(ioc) == 40:    # SHA1 hash
            driver.get(f"https://www.virustotal.com/gui/file/{ioc}")
        elif len(ioc) == 64:    # SHA256 hash
            driver.get(f"https://www.virustotal.com/gui/file/{ioc}")
        else:
            print(f"Error processing {ioc} ... skipping.")
        
        driver.implicitly_wait(3)   # wait for 3 seconds to not overload VT website
        count += 1
        details = extract_vt_details(driver, ioc)
        results.append(details)
    
    print(f"--- All {len(iocs)} IOCs have been checked in VirusTotal, check new Chrome windows for details. ---")

    # Display simple results
    print("\n" + "="*80)
    print("VIRUSTOTAL ANALYSIS RESULTS")
    print("="*80)
    
    for result in results:
        print(result)
    
    return results


# get ioc file
Base_dir = os.getcwd()
Filename = os.path.join(Base_dir, "iocs.txt")

# run the script
ioc_list = load_iocs(Filename)
results = run(ioc_list)

Loaded 8 IOCs.
Processing 1/8
Processing 2/8
Processing 3/8
Processing 4/8
Processing 5/8
Processing 6/8
Processing 7/8
Processing 8/8
--- All 8 IOCs have been checked in VirusTotal, check new Chrome windows for details. ---

VIRUSTOTAL ANALYSIS RESULTS
104.194.222.219 - 1/94 detections!
141.95.22.153 - 2/94 detections!
devsetgroup.com - 15/94 detections!
vsexec.com - 14/94 detections!
ec8952dc14bac73174cef02a489539e244b378b7de76c771126a8ba7ce532efd - 57/68 detections!
d1ba6260e2c6bf82be1d6815e19a1128aa0880f162a0691f667061c8fe8f1b2c - 40/65 detections!
08d40c504500c324b683773b1c6189d9 - 35/65 detections!
27f882a2b795abfae8f33440afcd3ad4 - 40/68 detections!
